In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib agg
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import jax
import jax.numpy as jnp
import visualization
from utils import results, experiments, settings, equioutput, evaluation

In [2]:
result = results.ResultSample.load_from_file(
    # 1 hl, 3 hn
    #"/home/gw/data/experiments/master_thesis/results/1cd0fb4d70b0c0c9bc850afe49a45128.tar.gz"
    #"/home/gw/data/experiments/master_thesis/results/cbc8fa72832a535540fd3a72274bd1df.tar.gz"
    #"/home/gw/data/experiments/master_thesis/results/f542d3ebd37584291a4d8e9cef5b95f7.tar.gz"
    
    # 1 hl, 4 hn
    #"/home/gw/data/experiments/master_thesis/results/e99343eb8d643d1e9212b2fcab193df5.tar.gz"
    #"/home/gw/data/experiments/master_thesis/results/cbc8fa72832a535540fd3a72274bd1df.tar.gz"
    #"/home/gw/data/experiments/master_thesis/results/c6835cefcab68b58b6f07158f1137c83.tar.gz"
    
    # 2 hl, 3 hn
    #"/home/gw/data/experiments/master_thesis/results/6fad5c77c6e4fcd72601720d57bf25fb.tar.gz"
    #"/home/gw/data/experiments/master_thesis/results/89478192c4d7fb7c0c138a6635f0ff04.tar.gz"
    #"/home/gw/data/experiments/master_thesis/results/3f20a27ee0828d865a36320f2af66afb.tar.gz"
    
    # 2 hl, 4 hn
    #"/home/gw/data/experiments/master_thesis/results/5fe59dd56f3ecc657fa49ac8b4ec4dce.tar.gz"
    #"/home/gw/data/experiments/master_thesis/results/f47e4647c414e39d47e2d5b79a59ce1e.tar.gz"
    #"/home/gw/data/experiments/master_thesis/results/ee1191f6065b8f88c88a62cffa269857.tar.gz"
    
    # 2 hl, 8hn
    #"/home/gw/data/experiments/master_thesis/results/37bccc0afc165ff842fbbeab926ab2f3.tar.gz"
    #"/home/gw/data/experiments/master_thesis/results/2afe0c254c0afa9dd1ba436f4be5affb.tar.gz"
    #"/home/gw/data/experiments/master_thesis/results/52a86ebd73aaa81538d2dccf80bd1b68.tar.gz"
    
    # 4 hl, 4 hn
    #"/home/gw/data/experiments/master_thesis/results/1664a41abb8c2042a3a316cefd1c5e87.tar.gz"
)
experiment = experiments.FactoryExperiment(result.experiment_type, **{"settings": result.settings})()
print(experiment._settings)

normalized probabilities: [0.5665243  0.3504857  0.08298998]
SettingsExperimentSample(output_path='/home/gw/data/experiments/master_thesis/results', dataset='regression2d', dataset_normalization='standardization', hidden_layers=1, hidden_neurons=4, activation='tanh', activation_last_layer='none', num_warmup=1024, statistic='reduced', statistic_p=0.99, samples_per_chain=1, identifiable_modes=3, pool_size=8, seed=0)


In [3]:
posterior_samples = result.samples["parameters"]

In [4]:
values, vectors = evaluation.full_pca(posterior_samples)
print(values.sum())

47.26016


/home/gw/data/binaries/mt/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:1865: ComplexWarning: Casting complex values to real discards the imaginary part
  out = lax_internal._convert_element_type(out, dtype, weak_type=weak_type)


In [5]:
sequential_helper = equioutput.SequentialHelper(
    transformation=experiment._model_transformation,
    dataset=experiment._dataset
)
structured_sequential_samples_parameters = sequential_helper.structured_sequential_samples_parameters(
    samples_parameters=result.samples["parameters"]
)
symmetry_helper = equioutput.SymmetryHelper(structured_sequential_samples_parameters)

print(result.samples["parameters"].shape)
print(sequential_helper._parameters_shapes)

(1274, 17)
FrozenDict({
    params: {
        layers_0: {
            bias: DeviceArray([4], dtype=int32),
            kernel: DeviceArray([2, 4], dtype=int32),
        },
        layers_2: {
            bias: DeviceArray([1], dtype=int32),
            kernel: DeviceArray([4, 1], dtype=int32),
        },
    },
})


In [6]:
for i in [0]:
    symmetry_helper.remove_tanh_symmetries(layer=i, rng_key=jax.random.PRNGKey(0))
    symmetry_helper.remove_permutation_symmetries(layer=i, iterations=128)

epoch: 0,  loss: 2.054232120513916, normal l2 norm: 1.0221617221832275
epoch: 1,  loss: 1.8301595449447632, normal l2 norm: 0.9761782288551331
epoch: 2,  loss: 1.6832115650177002, normal l2 norm: 1.397130012512207
epoch: 3,  loss: 1.6482349634170532, normal l2 norm: 1.2514084577560425
epoch: 4,  loss: 1.6039671897888184, normal l2 norm: 0.9817870259284973
epoch: 5,  loss: 1.5759882926940918, normal l2 norm: 0.8910833597183228
epoch: 6,  loss: 1.5478301048278809, normal l2 norm: 0.9499236345291138
epoch: 7,  loss: 1.5626332759857178, normal l2 norm: 1.0002843141555786
epoch: 8,  loss: 1.5348387956619263, normal l2 norm: 1.0320767164230347
epoch: 9,  loss: 1.5299197435379028, normal l2 norm: 0.9970718026161194
epoch: 10,  loss: 1.5232568979263306, normal l2 norm: 0.9320270419120789
epoch: 11,  loss: 1.5157103538513184, normal l2 norm: 1.0156810283660889
epoch: 12,  loss: 1.5096197128295898, normal l2 norm: 1.0241230726242065
epoch: 13,  loss: 1.5054455995559692, normal l2 norm: 0.9987402

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5096/5096 [00:21<00:00, 236.82it/s]


0 1617
1 1147
2 836
3 694
4 568
5 435
6 371
7 321
8 304
9 263
10 224
11 216
12 216
13 162
14 159
15 141
16 138
17 137
18 131
19 143
20 119
21 122
22 108
23 103
24 81
25 66
26 69
27 67
28 69
29 64
30 62
31 56
32 54
33 46
34 33
35 30
36 32
37 32
38 22
39 16
40 15
41 15
42 17
43 21
44 24
45 33
46 29
47 28
48 24
49 23
50 17
51 19
52 22
53 22
54 16
55 13
56 8
57 3
58 1
59 1
60 1
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 0
108 0
109 0
110 0
111 0
112 0
113 0
114 0
115 0
116 0
117 0
118 0
119 0
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 0


In [7]:
posterior_samples_relabeled = symmetry_helper._structured_sequential_samples_parameters.samples_parameters
values, vectors = evaluation.full_pca(posterior_samples_relabeled)
print(values.sum())

20.18863
